<a href="https://colab.research.google.com/github/Swayamprakashpatel/DE_Interact.github.io/blob/main/DE_Interact_WebLink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# @title Default title text
import tensorflow as tf
import pandas as pd
import numpy as np
import io
from tensorflow import keras
import pubchempy as pcp
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import time as tm
import gdown
import gradio as gr

# --- Model Loading and Setup ---
file_id = '13BpkcW3IRhbVWAVHK7JPwtB7WCSyG73W'
destination = '/content/sample_data/Model.hdf5'

gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=True)
model = load_model(destination, compile=False)

Y1 = np.array([['Possibally Compatible'], ['Possibally Incompatible']])
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y1)

# --- Core Prediction Function ---
def predict_compatibility(drug_cid, excipient_cid):
    """
    Predicts the compatibility of a drug and an excipient based on their PubChem CIDs.
    """
    try:
        CID_D = pcp.Compound.from_cid(drug_cid)
        CID_E = pcp.Compound.from_cid(excipient_cid)

        FPD = CID_D.cactvs_fingerprint
        FPE = CID_E.cactvs_fingerprint

        List = list(FPD) + list(FPE)
        t = pd.DataFrame(np.array(List).reshape(-1, len(List)))
        X_Predict = t.values[:, 0:1762].astype(int)

        t1 = model.predict(X_Predict)
        t2 = np.round(t1)
        Y_prediction = enc.inverse_transform(t2)

        if Y_prediction[0][0] == 'Possibally Incompatible':
            probability = t1[0][1]
            return f"❌ Incompatible (Probability: {probability:.2f})"
        else:
            probability = t1[0][0]
            return f"✅ Compatible (Probability: {probability:.2f})"

    except Exception as e:
        return f"🚨 Error: Could not process CIDs. Please check if they are valid. ({e})"

# --- Gradio Interface with Flagging Disabled ---
# The 'allow_flagging="never"' parameter disables the Flag button.
iface = gr.Interface(
    fn=predict_compatibility,
    inputs=[
        gr.Number(label="Drug PubChem Compound ID (CID)"),
        gr.Number(label="Excipient PubChem Compound ID (CID)")
    ],
    outputs=gr.Textbox(label="Prediction Result"),
    title="Drug-Excipient Compatibility Predictor",
    description="Enter the PubChem CIDs for a drug and an excipient to predict their compatibility using a neural network model.",
    allow_flagging="never"  # 👈 This line disables the Flag button
)

iface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://66918c56b3b3e0571c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
